# TO DO: put this in run script

In [1]:
import numpy as np
import torch
from mcspace.model import MCSPACE
from mcspace.trainer import train_model
from mcspace.data_utils import get_data, get_mouse_diet_perturbations_dataset
from mcspace.utils import get_device, pickle_load, pickle_save,\
    estimate_process_variance, MODEL_FILE, DATA_FILE, \
    get_mcspace_cooccur_prob, get_sig_perturbation_diff_subset, \
    get_posterior_summary_data, get_pert_otu_sub, get_min_loss_path, apply_taxonomy_threshold, \
    get_summary_results
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import mcspace.visualization as vis
from mcspace.dataset import DataSet
import pandas as pd

In [2]:
pwd

'C:\\Users\\Gary2\\Partners HealthCare Dropbox\\Gurdip Uppal\\research_bwh\\MCSPACE_FINAL\\MCSPACE\\mcspace\\paper\\mouse_runs'

# Paths

In [3]:
rootpath = Path("../../")
basepath = Path("./")

In [4]:
runpath = rootpath / "paper" / "mouse_runs" / "runs"

In [5]:
outpath = basepath / "results"
outpath.mkdir(exist_ok=True, parents=True)

# Load best model

In [6]:
respath = get_min_loss_path(runpath, np.arange(10))

model = torch.load(respath / MODEL_FILE)
data = pickle_load(respath / DATA_FILE)

taxonomy = pickle_load(respath / "taxonomy.pkl")

times = list(data['count_data'].keys())
subjects = list(data['count_data'][10].keys())
num_otus = data['count_data'][times[0]][subjects[0]].shape[1]
num_times = len(times)
num_subjects = len(subjects)

0
1
2
3
4
5
6
7
8
9
4


In [7]:
taxonomy = apply_taxonomy_threshold(taxonomy)

# do beforehand..
name_updates = {'Otu10': {'Species': 'Faecalibaculum rodentium'},
'Otu17': {'Genus': 'Roseburia'},
'Otu6': {'Species': 'Eubacterium coprostanoligenes'},
'Otu20': {'Species': 'Muribaculum gordoncarteri'},
'Otu15': {'Genus': 'Eisenbergiella'},
'Otu43': {'Family': 'Lachnospiraceae'}}

taxonomy['Species'] = 'na'
for oidx in name_updates.keys():
    replace = name_updates[oidx]
    key = list(replace.keys())[0]
    newname = replace[key]
    taxonomy.loc[oidx,key] = newname

# Get summary results

In [8]:
thetadf, betadf, pertsdf = get_posterior_summary_data(model, data, taxonomy, times, subjects)

# Output results

In [12]:
thetadf.to_csv(outpath / "assemblages.csv")
betadf.to_csv(outpath / "assemblage_proportions.csv")
pertsdf.to_csv(outpath / "perturbation_bayes_factors.csv")

# TODO: put in utils instead

In [16]:
import mcspace.vis_tools as vis

In [17]:
radf = vis.get_relative_abundances(data, times, subjects, taxonomy)

In [19]:
# radf

In [20]:
radf.to_csv(outpath / "relative_abundances.csv")